### Get jobs from API and put into MongoDB

You have to input your real secret and public key here

In [1]:
import upwork
pk = "YYY"
sk = "XXX"
client = upwork.Client(public_key = pk, secret_key = sk)
#category2
client.auth.get_request_token()

('8fdd02b1c550b2bc75af35aa5a268bca', '8737f87abb40b45f')

In [2]:
import requests

authorize_url = client.auth.get_authorize_url()

# Invoke the authorize url
requests.get(authorize_url)

verifier = raw_input(
    'Please enter the verification code you get '
    'following this link:\n{0}\n\n> '.format(
        client.auth.get_authorize_url()))

access_token, access_token_secret = client.auth.get_access_token(verifier)

Please enter the verification code you get following this link:
https://www.upwork.com/services/api/auth?oauth_token=8fdd02b1c550b2bc75af35aa5a268bca

> 344eae9bfa367e7c59ee2f47f69168da


In [156]:
from pymongo import MongoClient
client = MongoClient()
db = client.data_science_jobs
import json

In [ ]:
client = upwork.Client(pk, sk, access_token, access_token_secret)

In [13]:
def gather_jobs(page_number_start, page_number_end):
    data = {'category2':'Data Science & Analytics', 'q': ''}
    for i in range(page_number_start, page_number_end):
        # get 100 data scientists
        jobs = client.provider_v2.search_jobs(data=data, page_offset = (i - 1) * 99 + 1, page_size = 99)
        #if no entries returned, stop 
        if len(jobs) == 0: 
            break
        # insert records into DB    
        db.data_science_jobs.insert_many(jobs)

In [17]:
gather_jobs(2, 100)

In [23]:
len(db.data_science_jobs.distinct("url"))

3016

In [24]:
print "Gathered the %d jobs currently posted on Upwork" %len(db.data_science_jobs.distinct("url"))

Gathered the 3016 jobs currently posted on Upwork


### Organize job information from API

In [25]:
db.data_science_jobs.find_one().keys()

[u'job_status',
 u'category2',
 u'title',
 u'skills',
 u'job_type',
 u'budget',
 u'snippet',
 u'url',
 u'workload',
 u'client',
 u'subcategory2',
 u'duration',
 u'date_created',
 u'_id',
 u'id']

We want title, skills, job_type, budget, snippet, workload, client, duration, and id

In [29]:
import pandas as pd

In [222]:
cursor = db.data_science_jobs.find({}, {"title": 1, "skills": 1, 
         "id": 1, "job_type": 1, "budget": 1, "snippet": 1, 
          "workload": 1, "client": 1, "duration": 1, "url": 1})
jobs = pd.DataFrame(list(cursor))

## Make Skill table jobs

In [162]:
job_skills = jobs[["id", "skills"]]

In [163]:
job_skills.head()

,id,skills
0,~01843684906136dc42,"[excel-vba, javascript, microsoft-excel]"
1,~013e5a61f2dd228aba,"[data-entry, data-scraping]"
2,~015369c631ecbb94df,[]
3,~01f11a2bb87e06eb26,[]
4,~014e134eb51da5da9d,"[web-scraping, wordpress]"


In [164]:
splitted = job_skills['skills'].apply(lambda x: pd.Series(list(x)))
wide_skills = pd.concat([job_skills, splitted], axis=1)
del wide_skills["skills"]
molten = pd.melt(wide_skills, id_vars = "id", value_name = "skill")
molten = molten.dropna()
del molten["variable"]
molten.head()

,id,skill
0,~01843684906136dc42,excel-vba
1,~013e5a61f2dd228aba,data-entry
4,~014e134eb51da5da9d,web-scraping
6,~01e2ca0de451c40a7b,algorithm-development
7,~01e545de4066feed8e,java


## Unnest client information 

In [223]:
def get_client_information(df):
    # initialize dataframe
    new_df = pd.DataFrame(df["client"].iloc[0], index = [0])
    for i in range(1, len(client_jobs)):
        new_df = new_df.append(pd.DataFrame(client_jobs["client"].iloc[i], index = [i]))
    return new_df
        

In [224]:
job_clients = get_client_information(jobs)

In [225]:
jobs = jobs.join(job_clients)
del jobs["client"]
del jobs["_id"]

In [227]:
pd.DataFrame.to_csv(jobs[["id", "url", "reviews_count"]], "jobs_url.csv")

In [226]:
jobs.head()

,budget,duration,id,job_type,skills,snippet,title,url,workload,country,feedback,jobs_posted,past_hires,payment_verification_status,reviews_count
0,27.0,None,~01843684906136dc42,Fixed,"[excel-vba, javascript, microsoft-excel]",We are looking for someone to create a VBA exc...,VBA / Macros for Geocoding Addresses,http://www.upwork.com/jobs/~01843684906136dc42,30+ hrs/week,United States,2.934960,57,38,VERIFIED,28
1,0.0,3 to 6 months,~013e5a61f2dd228aba,Hourly,"[data-entry, data-scraping]",Looking for someone to do data collection.\n\n...,Need someone for data collection.,http://www.upwork.com/jobs/~013e5a61f2dd228aba,Less than 10 hrs/week,United States,4.482988,62,42,VERIFIED,28
2,0.0,Less than 1 week,~015369c631ecbb94df,Hourly,[],Please read the entire post before responding:...,South Florida Realtor Email List Wanted,http://www.upwork.com/jobs/~015369c631ecbb94df,Less than 10 hrs/week,United States,5.000000,34,15,VERIFIED,1
3,0.0,Less than 1 week,~01f11a2bb87e06eb26,Hourly,[],Please read the entire post before responding:...,South Florida Realtor Email List Wanted,http://www.upwork.com/jobs/~01f11a2bb87e06eb26,Less than 10 hrs/week,United States,5.000000,34,15,VERIFIED,1
4,0.0,More than 6 months,~014e134eb51da5da9d,Hourly,"[web-scraping, wordpress]",We are looking for someone who can scrap one w...,Scraping website and uploading to Wordpress,http://www.upwork.com/jobs/~014e134eb51da5da9d,30+ hrs/week,Canada,0.000000,3,0,VERIFIED,0


In [168]:
jobs["snippet"] = jobs["snippet"].apply(lambda x: x.replace("\n", ""))

In [196]:
with open('jobs.pkl', 'w') as picklefile:
    pickle.dump(jobs, picklefile)

In [152]:
count_chars = jobs["snippet"].apply(lambda x: len(x))
print "We have %d jobs that have more than 250 characters" %len(count_chars[count_chars > 250])

We have 1963 jobs that have more than 250 characters


In [181]:
# encoding in unicode
jobs["snippet"] = jobs["snippet"].apply(lambda x: x.encode('ascii', 'ignore'))
jobs["snippet"] = jobs["snippet"].apply(lambda x: x.replace("\r", ""))
jobs["title"] = jobs["title"].apply(lambda x: x.replace("\r", ""))
jobs["title"] = jobs["title"].apply(lambda x: x.encode('ascii', 'ignore'))

In [220]:
jobs.to_csv("jobs.csv")